### Practical QR and Singular Value Decomposition (SVD)

In the first part, I have implemented a QR factorization method that uses Householder's reflections (that are unitary operators) to upper-diagonalize a given matrix.

In the second part, I have implemented a fast SVD algorithm. It first utilize Householder's reflections to make the given matrix bidiagonal and then run a series of implicit QR iterations until the given matrix converges to a diagonal matrix as desired. 

In [6]:
import numpy as np
import math

EPS = 1e-10 # Maximum precision of the calculation
TOL = 1e-15 # Maximum precision of the machine
MAX_ITERATION = 100 # Maximum number of iterations in SVD algorithm

def get_sign(x):
    '''returns sign of x'''
    if x >= 0:
        return 1
    return -1

def householder_qr(A):
    '''returns QR factorization of A as (Q, R)'''
    (nrow, ncol) = np.shape(A)
    Q = np.identity(nrow, dtype=np.float128)
    R = np.copy(A)
    for row in range(nrow):
        x = R[row:nrow, row]
        
        # y is the desired reflection of x
        y = np.zeros_like(x, dtype=float)
        y[0] = -get_sign(x[0]) * np.linalg.norm(x)
        
        # u is the householder vector for desired reflection transform
        u = np.zeros(nrow)
        u[row:nrow] = y - x
        u = u / np.linalg.norm(u)
        
        H = np.identity(nrow, dtype=np.float128) - 2*np.outer(u, u)
        Q = Q @ H
        R = H @ R
    return (Q, R)

In [7]:
def householder_bidiagonalization(A):
    '''returns Bidiagnoliziation of A as (U,A,V*)'''
    # For now, I assume ncol <= nrow
    
    (nrow, ncol) = np.shape(A)
    U = np.identity(nrow, dtype=np.float128)
    V = np.identity(ncol, dtype=np.float128)
    R = np.copy(A)
    
    col = 0
    for row in range(nrow):
        x = R[row:nrow, row]
        
        # y is the desired reflection of x
        y = np.zeros_like(x, dtype=float)
        y[0] = -get_sign(x[0]) * np.linalg.norm(x)

        # u is the householder vector for desired reflection transform
        u = np.zeros(nrow)
        u[row:nrow] = y - x
        u = u / np.linalg.norm(u)
        
        H = np.identity(nrow, dtype=np.float128) - 2*np.outer(u, u)
        U = U @ H
        R = H @ R
        
        if row+1 < ncol:
            x = R[row, row+1:ncol]

            # y is the desired reflection of x
            y = np.zeros_like(x, dtype=float)
            y[0] = -get_sign(x[0]) * np.linalg.norm(x)

            # u is the householder vector for desired reflection transform
            u = np.zeros(ncol)
            u[row+1:ncol] = y - x
            u = u / np.linalg.norm(u)

            H = np.identity(nrow, dtype=np.float128) - 2*np.outer(u, u)

            V = H @ V
            R = R @ H
    
    for row in range(nrow):
        for col in range(ncol):
            if abs(R[row, col]) < EPS: R[row, col] = 0.0
                
    return (U, R, V)


def blocking(B):
    ''' B must be a square upper bidiagonal matrix'''
    n = len(B)

    # identify size of larget diagonal sub matrix 
    q = 0
    if abs(B[n-2, n-1]) < EPS:
        q = 1
    while q > 0 and q < n:
        if n-q-2>=0 and abs(B[n-q-2, n-q-1]) > EPS: 
            break
        q = q + 1

    # identify larget submatrix with non-zero superdiagonal entries.
    p = n - q
    while p > 0:
        if p-2>=0 and abs(B[p-2, p-1]) < EPS:
            p = p - 1
            break;
        p = p-1

    B_1 = B[0:p, 0:p]
    B_2 = B[p:n-q, p:n-q]
    B_3 = B[n-q:, n-q:]
    return (B_1, B_2, B_3)


def wilkinson_2by2(C):
    '''Extract an approximate Wilkinson shift for 2 by 2 matrix C'''

    delta = (C[0,0]+C[1,1])**2 - 4*(C[0,0]*C[1,1]-C[0,1]*C[1,0])
    if delta > 0:
        lambda_1 = ((C[0,0]+C[1,1]) + math.sqrt(delta))/2
        lambda_2 = ((C[0,0]+C[1,1]) - math.sqrt(delta))/2
        if abs(lambda_1 - C[1,1]) < abs(lambda_2 - C[1,1]):
            return lambda_1
        return lambda_2
    return C[1,1]


def givens(i, j, _n, x, y):
    ''' 
        Returns a Givens rotation matrix with size n 
        The rotation zero out j-th row for a column c that c[i] = x, c[j] = y
    '''
    G = np.identity(_n, dtype=np.float128)
    norm = math.sqrt(x**2 + y**2)
    c = x / norm
    s = y / norm
    G[i, i], G[i, j], G[j, i], G[j, j] = c, s, -s, c

    return G
    
    
def svd(A):
    ''' For now I assume that A is a square matrix'''
    (nrow, ncol) = np.shape(A)
    (U, B, V) = householder_bidiagonalization(A)
    n = len(B)

    for iteration in range(MAX_ITERATION):
        # Zero out small superdiagonal entries
        for i in range(n-1):
            if abs(B[i, i+1]) < EPS*(abs(B[i, i])+abs(B[i+1, i+1])):
                B[i, i+1] = 0.0
        
        # Zero out small entries
        for i in range(n):
            for j in range(n):
                if abs(B[i, j]) < TOL:
                    B[i, j] = 0.0
      
        # Split B into blocks
        (B_1, B_2, B_3) = blocking(B)
        
        # We have found the decomposition!
        if len(B_3) == n:
            return (U, B, V)
        
        zero_diagonal_entry = False
        for i in range(len(B_2)-1):
            if B_2[i, i] == 0:
                ''' We use Givens rotations to zero out the upperdiagonal entry in i-th row'''
                U_2 = np.identity(len(B_2), dtype=np.float128)
                for j in range(i, len(B_2)-1):
                    G = givens(j+1, j, n, B[i+1, i+1], B_2[i, i+1])
                    B_2 = G @ B_2
                    U_2 = U_2 @ G.T

                U[len(B_1):len(B_1)+len(B_2), len(B_1):len(B_1)+len(B_2)] = \
                                U[len(B_1):len(B_1)+len(B_2), len(B_1):len(B_1)+len(B_2)] @ U_2
                zero_diagonal_entry = True
                break
        
        if not zero_diagonal_entry:
            ''' If we reach here, we are sure that B_2 has no zero diagonal or superdiagonal entry'''
            n_2 = len(B_2)
            
            U_2 = np.identity(n_2, dtype=np.float128)
            V_2 = np.identity(n_2, dtype=np.float128)
            
            mu = wilkinson_2by2(B_2[n_2-2:, n_2-2:].T @ B_2[n_2-2:, n_2-2:])

            y = B_2[0,0]**2 - mu
            z = B_2[0,0]*B_2[0, 1]
            for i in range(0, n_2-1):
                G = givens(i, i+1, n_2, y, z)

                B_2 = B_2 @ G.T
                V_2 = G @ V_2
                
                y, z = B_2[i,i], B_2[i+1, i]
                G = givens(i, i+1, n_2, y, z)
                B_2 = G @ B_2
                U_2 = U_2 @ G.T
                        
                if i < n_2-2:
                    y, z = B_2[i,i+1], B_2[i,i+2]
            
        B[len(B_1):len(B_1)+len(B_2), len(B_1):len(B_1)+len(B_2)] = B_2
        U_2_extended = np.identity(n)
        V_2_extended = np.identity(n)
        U_2_extended[len(B_1):len(B_1)+len(B_2), len(B_1):len(B_1)+len(B_2)] = U_2
        V_2_extended[len(B_1):len(B_1)+len(B_2), len(B_1):len(B_1)+len(B_2)] = V_2
        
        U = U@U_2_extended
        V = V_2_extended@V
        # END OF ITERATION
            
    return (U, B, V)

In [8]:
### TEST QR Factorization

def sum_under_diagonal(R): 
    result = 0
    for i in range(len(R)): 
        for j in range(len(R[i])):
            if i > j:
                result += R[i,j]
    return result


A = np.matrix([[1.12, 2.1, 3, 4],
               [5, 0.2, 7, 8],
               [23, 3, 2, 1],
               [8, 7, 6, 12]])

Q, R = householder_qr(A)

                
print("Total error upper diagonal R:", sum_under_diagonal(R))
print("Total error unitary Q:", (Q.T @ Q - np.identity(len(A))).sum())
print("Total reconstruction error: ", (A - Q@R).sum())

Total error upper diagonal R: 4.661709887976587833e-15
Total error unitary Q: 4.905011096454411218e-16
Total reconstruction error:  -4.588838261199851276e-14


In [9]:
### TEST Bidiagonalization

A = np.matrix([[1.12, 2.1, 3, 4],
               [5, 0.2, 7, 8],
               [23, 3, 2, 1],
               [8, 7, 6, 12]])


(U, B, V) = householder_bidiagonalization(A)

print("Total error upper bidiagonal B:", sum_under_diagonal(B)+sum_under_diagonal(B[:-1,1:].T))
print("Total error unitary U:", (U.T @ U - np.identity(len(B))).sum())
print("Total error unitary V:", (V.T @ V - np.identity(len(B))).sum())
print("Total reconstruction error: ", (A - U@B@V).sum())

print("------------")

A = np.matrix([[1.12, 2.1, -1233, 4],
               [5, 0.2, 123123, 8],
               [12312, 3, 2, 1],
               [8, 7, 6, 12]], dtype=np.float128)

(U, B, V) = householder_bidiagonalization(A)

print("Total error upper bidiagonal B:", sum_under_diagonal(B)+sum_under_diagonal(B[:-1,1:].T))
print("Total error unitary U:", (U.T @ U - np.identity(len(B))).sum())
print("Total error unitary V:", (V.T @ V - np.identity(len(B))).sum())
print("Total reconstruction error: ", (A - U@B@V).sum())



Total error upper bidiagonal B: 0.0
Total error unitary U: -7.2127904777313789353e-16
Total error unitary V: 1.4441572937506919061e-16
Total reconstruction error:  -4.3729343063292347438e-14
------------
Total error upper bidiagonal B: 0.0
Total error unitary U: -2.7617618568776638802e-17
Total error unitary V: 9.028236425192092367e-16
Total reconstruction error:  1.05609893931175174986e-11


In [10]:
### TEST Blocking 

test1 = np.identity(4)
B_1, B_2, B_3 = blocking(test1)
print(B_1)
print(B_2)
print(B_3)

print("--------------")

test1[0, 1] = 1
B_1, B_2, B_3 = blocking(test1)
print(B_1)
print(B_2)
print(B_3)

print("--------------")

test1[0, 1] = 0
test1[1, 2] = 1
B_1, B_2, B_3 = blocking(test1)
print(B_1)
print(B_2)
print(B_3)

print("--------------")

test1[0, 1] = 1
B_1, B_2, B_3 = blocking(test1)
print(B_1)
print(B_2)
print(B_3)

[]
[]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
--------------
[]
[[1. 1.]
 [0. 1.]]
[[1. 0.]
 [0. 1.]]
--------------
[[1.]]
[[1. 1.]
 [0. 1.]]
[[1.]]
--------------
[]
[[1. 1. 0.]
 [0. 1. 1.]
 [0. 0. 1.]]
[[1.]]


In [11]:
### TEST Wilkinson shift

wilkinson_2by2(np.matrix([[1, 2],[3, 4]]))

5.372281323269014

In [12]:
### TEST Givens rotations

x = np.matrix([1, 2, 3, 4]).T
G = givens(0, 1, 4, 1, 2)

G @ x

matrix([[2.23606798],
        [0.        ],
        [3.        ],
        [4.        ]], dtype=float128)

In [13]:
# TEST SVD

X = np.matrix([[1.12, 2.1, -1233, 4],
               [5, 0.2, 123123, 8],
               [12312, 3, 2, 1],
               [8, 7, 6, 12]], dtype=np.float128)

(U, S, V) = svd(X)


print("Total error unitary U:", (U.T @ U - np.identity(len(B))).sum())
print("Total error unitary V:", (V.T @ V - np.identity(len(B))).sum())
print("Total error diagonal S:", sum_under_diagonal(S)+sum_under_diagonal(S.T))
print("Total decomposition error:", np.abs(X - U @ S @ V).sum())

Total error unitary U: 1.1778989675346067943e-15
Total error unitary V: 1.8947457058041540166e-15
Total error diagonal S: 0.0
Total decomposition error: 1.0875094439343965802e-10
